In [ ]:
import keras
import pandas as pd
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

In [ ]:
import keras
import tensorflow as tf
import segmentation_models as sm
from data_generator import DataGenerator
from segmentation_models.metrics import IOUScore, FScore
from segmentation_models.losses import DiceLoss, BinaryFocalLoss


sm.set_framework('tf.keras')
sm.framework()

# setting dynamic memory growth for GPU VRAM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
# Defining Hyper-parameters

BACKBONE = 'efficientnetb3'
CLASSES = ['floor']
LR = 0.0001
EPOCHS = 10
batch_size = 2
input_shape=(128, 128, 3) # divisible by 32

preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
# Defining network parameters
n_classes = 1
activation = 'sigmoid' 

In [ ]:
# Create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation, input_shape=input_shape)

# Define optomizer
optim = tf.keras.optimizers.Adam(LR)

# Combining segmentation model losses
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# Compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

In [ ]:
# Creating data generators for train and valid

train_df = pd.read_csv( 'train.csv')
train_gen = DataGenerator(train_df,
                          X_col='rgb',
                          y_col='mask',
                          batch_size=batch_size, 
                          input_size=input_shape,
                          shuffle=True)

valid_df  = pd.read_csv('valid.csv')
valid_gen  = DataGenerator(valid_df,
                          X_col='rgb',
                          y_col='mask',
                          batch_size=batch_size,
                          input_size=input_shape,
                          shuffle=False)


In [ ]:
# Loading sample from data generator

X, y = valid_gen.get_sample(0)

# Image
plt.imshow(X)
plt.show()
# Ground truth
plt.imshow(y)
plt.show()

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", restore_best_weights=True, mode='min', patience=15, verbose=1),
    keras.callbacks.ModelCheckpoint('{val_loss:.4f}.h5', monitor="val_loss", mode="min", save_best_only=True, save_weights_only=False,  verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='min', min_delta=0.0001, cooldown=0, min_lr=0)
]

In [ ]:
# train the model
history = model.fit(
    train_gen,  
    epochs=EPOCHS,
    callbacks=callbacks, 
    validation_data=valid_gen,
    batch_size=batch_size
)